In [207]:
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
import re
from itertools import * 


Extract all request parameters

In [22]:
html_doc = urllib.request.urlopen("http://isaww_i_reportModelXsl.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247").read()
soup = BeautifulSoup(html_doc, 'html.parser')

In [325]:
OFFSET_GENDER = 0
OFFSET_NAME = 1
OFFSET_STATUS = 7
OFFSET_SCIPER = 10

# transform the iterable in none if it's empty
def peek(iterable):
    try:
        first = next(iterable)
    except StopIteration:
        return None
    return chain([first], iterable)

# return the request url for the given parameters
def buildRequest(params):
    requestParams = ''
    for key in params:
        requestParams += key + '=' + str(params[key]) + '&'
    return "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?" + requestParams


def parsePeriod(period):
    if "Bachelor semestre" in period:
        return ('Bachelor', period[len('Bachelor semestre '):])
    if "Master semestre" in period:
        return ('Master', period[len('Master semestre '):])

    # TODO: add support for master thesis and others
    
def parseResult(html):
    def genPersonDf(person):
        personDict = parsePerson(person)
        return pd.DataFrame(personDict, index=[personDict['sciper']])
        
    def parsePerson(person):
        elems = person.findChildren()
        return {
            'gender': elems[OFFSET_GENDER].next,
            'name': elems[OFFSET_NAME].next,
            'status': elems[OFFSET_STATUS].next,
            'sciper': elems[OFFSET_SCIPER].next,
        }
    soup = BeautifulSoup(html, 'html.parser')
    
    it = (genPersonDf(person) for person in soup.findAll('tr')[2:])
    it = peek(it)
    if it is None:
        return pd.DataFrame()
    else:
        return pd.concat(it)



def genDF(params):
    request = buildRequest({
                                'ww_x_GPS': ww_x_GPS,
                                'ww_i_reportModel': ww_i_reportModel,
                                'ww_i_reportModelXsl': ww_i_reportModelXsl,
                                'ww_x_PERIODE_ACAD': params[1]['value'],
                                'ww_x_UNITE_ACAD': params[2]['value'],
                                'ww_x_PERIODE_PEDAGO': params[0]['value'],
                                'ww_x_HIVERETE': params[3]['value']
                             })
    with urllib.request.urlopen(request) as c:
        html = c.read()
        df = parseResult(html)
        df['year'] = params[1].next
        df['section'] = params[2].next
        period = parsePeriod(params[0].next)
        df['level'] = period[0]
        df['semester'] = period[1]
        return df

# select all?
ww_x_GPS=-1

ww_i_reportModel =  soup.findAll("input", {"name": "ww_i_reportmodel"})[0]['value']

ww_i_reportModelXsl =  soup.findAll("input", {"name": "ww_i_reportModelXsl"})[0]['value'] # select the first value as it is the html report

# remove the null by doing removing the first element
ww_x_UNITE_ACADs =  soup.findAll("select", {"name": "ww_x_UNITE_ACAD"})[0].findChildren()[1:] 
ww_x_PERIODE_ACADs =  soup.findAll("select", {"name": "ww_x_PERIODE_ACAD"})[0].findChildren()[1:]
ww_x_PERIODE_PEDAGOs = soup.findAll("select", {"name": "ww_x_PERIODE_PEDAGO"})[0].findChildren()[1:]
ww_x_HIVERETEs = soup.findAll("select", {"name": "ww_x_HIVERETE"})[0].findChildren()[1:]


# ww_x_PERIODE_ACAD = ww_x_PERIODE_ACADs[0]
# ww_x_UNITE_ACAD = ww_x_UNITE_ACADs[0]
# ww_x_PERIODE_PEDAGO = ww_x_PERIODE_PEDAGOs[0]
# ww_x_HIVERETE = ww_x_HIVERETEs[0]

it = product(\
            filter(lambda x: "Bachelor" in x.next, ww_x_PERIODE_PEDAGOs),
            filter(lambda x: x.next == "2015-2016", ww_x_PERIODE_ACADs),
            filter(lambda x: x.next == "Architecture", ww_x_UNITE_ACADs),
            ww_x_HIVERETEs
           )
# for (x, y, z, a) in it:
#     print(x, y, z, a)
             
pd.concat(list(map(genDF, it)))

# pd.concat(map(genDF, it))
# list(map(genDF, it))


# list( (genDF(next(it)), genDF(next(it))) )
                
# sum(1 for _ in it)

# request =  buildRequest({
#                 'ww_x_GPS': ww_x_GPS,
#                 'ww_i_reportModel': ww_i_reportModel,
#                 'ww_i_reportModelXsl': ww_i_reportModelXsl,
#                 'ww_x_PERIODE_ACAD': ww_x_PERIODE_ACAD['value'],
#                 'ww_x_UNITE_ACAD': ww_x_UNITE_ACAD['value'],
#                 'ww_x_PERIODE_PEDAGO': ww_x_PERIODE_PEDAGO['value'],
#                 'ww_x_HIVERETE': ww_x_HIVERETE['value']
#              })
# with urllib.request.urlopen(request) as c:
#     html = c.read()
#     print(parseResult(html))

,gender,level,name,sciper,section,semester,status,year
257858,Madame,Bachelor,Abbé-Decarroux Maud Clara,257858,Architecture,1,Présent,2015-2016
259395,Monsieur,Bachelor,Abdalla Giovanni Ekram Boutros Salib,259395,Architecture,1,Présent,2015-2016
259921,Madame,Bachelor,Aellen Valentine Sarah,259921,Architecture,1,Présent,2015-2016
259923,Monsieur,Bachelor,Aeschimann Samuel Antoine,259923,Architecture,1,Présent,2015-2016
247772,Madame,Bachelor,Ali Noémie,247772,Architecture,1,Présent,2015-2016
259926,Madame,Bachelor,Alioth Lucie Audrey,259926,Architecture,1,Présent,2015-2016
250356,Madame,Bachelor,Allaz Pauline,250356,Architecture,1,Présent,2015-2016
259927,Madame,Bachelor,Ammeter Emma,259927,Architecture,1,Présent,2015-2016
259180,Monsieur,Bachelor,Antonietti Luciano Nicolas,259180,Architecture,1,Présent,2015-2016
258092,Madame,Bachelor,Armanet Juliette Maïa,258092,Architecture,1,Présent,2015-2016


In [205]:



# for ww_x_PERIODE_PEDAGO in ww_x_PERIODE_PEDAGOs:
#     print(parsePeriod(ww_x_PERIODE_PEDAGO.next))

,year,section,level,semester


In [193]:
ww_x_PERIODE_ACADs

[<option value="355925344">2016-2017</option>,
 <option value="213638028">2015-2016</option>,
 <option value="213637922">2014-2015</option>,
 <option value="213637754">2013-2014</option>,
 <option value="123456101">2012-2013</option>,
 <option value="123455150">2011-2012</option>,
 <option value="39486325">2010-2011</option>,
 <option value="978195">2009-2010</option>,
 <option value="978187">2008-2009</option>,
 <option value="978181">2007-2008</option>]

In [319]:
df = pd.DataFrame({'a': 1, 'b': 2}, index = [0])
df

pd.concat( (df, df)[1:2] )



,a,b
0,1,2
